Будем осуществлять работу с непростым набором данных о состоянии здоровья лошадей, испытывающих кишечные колики.

https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv


Задание 1. Базовое изучение
Изучить представленный набор данных на основе описания его столбцов и выбрать 8 столбцов для дальнейшего изучения
https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names
(среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

In [94]:
import pandas as pd

import numpy as np

content = pd.read_csv('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv', names = ['surgery?', 'Age', 'Hospital Number', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'peripheral pulse', 'mucous membranes', 'capillary refill time', 'pain', 'peristalsis', 'abdominal distension', 'nasogastric tube', 'nasogastric reflux', 'nasogastric reflux PH', 'rectal examination', 'abdomen', 'packed cell volume', 'total protein','abdominocentesis appearance', 'abdomcentesis total protein', 'outcome', 'surgical lesion?', 'first number is site of lesion', 'second number is type', 'third number is subtype', 'cp_data']) # 'third number is subtype', 'fourth number is specific code', 'cp_data'])

content_selected = content[['surgery?', 'Hospital Number', 'rectal temperature', 'pulse', 'peripheral pulse', 'mucous membranes','pain', 'packed cell volume']]

content_selected = content_selected.replace('?', np.nan)

content_selected = content_selected.astype('float64')

content_selected.describe()


#content_selected.head(10)
#content_selected.info()



,surgery?,Hospital Number,rectal temperature,pulse,peripheral pulse,mucous membranes,pain,packed cell volume
count,299.000000,3.000000e+02,240.000000,276.000000,231.000000,253.000000,245.00000,271.000000
mean,1.397993,1.085889e+06,38.167917,71.913043,2.017316,2.853755,2.95102,46.295203
std,0.490305,1.529801e+06,0.732289,28.630557,1.042428,1.620294,1.30794,10.419335
min,1.000000,5.184760e+05,35.400000,30.000000,1.000000,1.000000,1.00000,23.000000
25%,1.000000,5.289040e+05,37.800000,48.000000,1.000000,1.000000,2.00000,38.000000
50%,1.000000,5.303055e+05,38.200000,64.000000,2.000000,3.000000,3.00000,45.000000
75%,2.000000,5.347275e+05,38.500000,88.000000,3.000000,4.000000,4.00000,52.000000
max,2.000000,5.305629e+06,40.800000,184.000000,4.000000,6.000000,5.00000,75.000000


Выбранные столбцы с типами значений:

1. surgery? - категориальный
2. Hospital Number - числовой
3. rectal temperature - числовой
4. pulse - числовой
5. peripheral pulse - категориальный
6. mucous membranes - категориальный
7. pain - категориальный
8. packed cell volume - числовой


Интерпретация результата расчета базовых метрик:
Значения rectal temperature различаются незначительно, но значения пульса различаются в несколько раз. Скорей всего последнее объясняется тем, что измерения проводятся как в ситуациях, когда лошадь долго находится в спокойном состоянии, так и сразу после длительной физической нагрузки. 

Исходя из описания в файле horse_data.names, можно сделать вывод, что большинство лошадей, по которым собирались данные, здоровы.




Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. 

Принять и обосновать решение о дальнейшей работе с ними.

Выбросы имеет смысл искать только по столбцам с числовыми значениями. 

Столбец "Hospital Number" выбросов иметь не может - это просто присвоенный лошади номер. 

Анализ базовых метрик показывает, что показатель 'rectal temperature' так же не имеет выбросов: минимальное значение мало отличается от максимального. 


Анализируем на выбросы столбцы 'pulse' и 'packed cell volume'. 

Выбросы находим в выборке, применяя стандартный подход с отбрасыванием значений, выходящих за пределы межквартильного размаха.


In [106]:
import pandas as pd

import numpy as np

content = pd.read_csv('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv', names = ['surgery?', 'Age', 'Hospital Number', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'peripheral pulse', 'mucous membranes', 'capillary refill time', 'pain', 'peristalsis', 'abdominal distension', 'nasogastric tube', 'nasogastric reflux', 'nasogastric reflux PH', 'rectal examination', 'abdomen', 'packed cell volume', 'total protein','abdominocentesis appearance', 'abdomcentesis total protein', 'outcome', 'surgical lesion?', 'first number is site of lesion', 'second number is type', 'third number is subtype', 'cp_data']) # 'third number is subtype', 'fourth number is specific code', 'cp_data'])

content_selected = content[['surgery?', 'Hospital Number', 'rectal temperature', 'pulse', 'peripheral pulse', 'mucous membranes','pain', 'packed cell volume']]

content_selected = content_selected.replace('?', np.nan)

content_selected = content_selected.astype('float64')

# Анализируем на выбросы столбец 'pulse':
pulse_blowout_analys = content_selected[['pulse', 'packed cell volume']]
q1 = pulse_blowout_analys['pulse'].quantile(0.25)
q3 = pulse_blowout_analys['pulse'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = pulse_blowout_analys[pulse_blowout_analys['pulse'].between(lower_bound, upper_bound, inclusive=True)].sort_values('packed cell volume')
result_with_nan = pd.concat([pulse_blowout_analys, remove_outliers]).drop_duplicates(keep=False)
final_result_pulse = result_with_nan['pulse'].dropna() 
final_result_pulse

# Анализируем на выбросы столбец 'packed cell volume':
pcv_blowout_analys = content_selected[['pulse', 'packed cell volume']]
q1 = pcv_blowout_analys['packed cell volume'].quantile(0.25)
q3 = pcv_blowout_analys['packed cell volume'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = pcv_blowout_analys[pcv_blowout_analys['packed cell volume'].between(lower_bound, upper_bound, inclusive=True)].sort_values('pulse')
result_with_nan = pd.concat([pcv_blowout_analys, remove_outliers]).drop_duplicates(keep=False)
final_result_pcv = result_with_nan['packed cell volume'].dropna() 

print(final_result_pulse)
print(final_result_pcv)

3      164.0
41     150.0
55     160.0
255    184.0
275    150.0
Name: pulse, dtype: float64
4      74.0
144    75.0
151    75.0
Name: packed cell volume, dtype: float64


Исходя из описаний в файле horse_data.names, можно сделать вывод, что выбросы являются естественными и их имеет смысл оставить в данных для дальнейшего анализа




Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. 

Принять и обосновать решение о методе работы с пропусками по каждому столбцу, 

сформировать датафрейм, в котором пропуски будут отсутствовать.

Поскольку мы не знаем о причинах появления пропусков, логичней всего принять решение об оставлении строк с пропусками для дальнейшего анализа и о том, чтобы не заменять пропуски на какие-то другие значения.

In [107]:
import pandas as pd

import numpy as np

content = pd.read_csv('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv', names = ['surgery?', 'Age', 'Hospital Number', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'peripheral pulse', 'mucous membranes', 'capillary refill time', 'pain', 'peristalsis', 'abdominal distension', 'nasogastric tube', 'nasogastric reflux', 'nasogastric reflux PH', 'rectal examination', 'abdomen', 'packed cell volume', 'total protein','abdominocentesis appearance', 'abdomcentesis total protein', 'outcome', 'surgical lesion?', 'first number is site of lesion', 'second number is type', 'third number is subtype', 'cp_data']) # 'third number is subtype', 'fourth number is specific code', 'cp_data'])

content_selected = content[['surgery?', 'Hospital Number', 'rectal temperature', 'pulse', 'peripheral pulse', 'mucous membranes','pain', 'packed cell volume']]

content_selected = content_selected.replace('?', np.nan)

# подсчет процента пустых значений по столбцам:
for col in content_selected.columns:
    pct_missing = content_selected[col].isnull().mean()
    print(f'{col} - {pct_missing :.1%}')
    
# датафрейм, в котором отсутствуют пропуски:
content_selected_without_skip = content_selected.dropna()    

content_selected_without_skip.head(10)

surgery? - 0.3%
Hospital Number - 0.0%
rectal temperature - 20.0%
pulse - 8.0%
peripheral pulse - 23.0%
mucous membranes - 15.7%
pain - 18.3%
packed cell volume - 9.7%


,surgery?,Hospital Number,rectal temperature,pulse,peripheral pulse,mucous membranes,pain,packed cell volume
2,2,530334,38.30,40,1,3,3,33.00
3,1,5290409,39.10,164,1,6,2,48.00
6,1,526802,37.90,48,1,1,3,37.00
10,1,528548,38.10,66,3,5,3,44.00
13,2,5291329,38.00,92,1,2,1,37.00
14,1,534917,38.2,76,1,1,3,46
15,1,530233,37.60,96,1,4,5,45.00
18,1,5290481,37.60,64,1,2,2,40.00
20,1,530157,39.90,72,1,5,5,46.00
22,1,521681,38.60,42,1,4,2,48.00
